# Write some words to MostUsedWord file

In [1]:
pwd

u'/Users/jshanahan/Dropbox/Lectures-UC-Berkeley-ML-Class-2015/Slides/Lecture-04-ClusteringInMrJob/MrJob/MrJobIntroJobs'

In [2]:
!echo foo foo quux labs foo bar quux > MostUsedWord.txt


## Append some more records to the file

In [1]:
!echo New line by Jimi >> MostUsedWord.txt
!echo New line by Jimi 2222222 >> MostUsedWord.txt


# Show the code of MrJob class for Most frequent/used Word

In [14]:
%%writefile Mostused.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+")

class MRMostUsedWord(MRJob):

#    OUTPUT_PROTOCOL = JSONValueProtocol
    SORT_VALUES = True
    def jobconf(self):
        orig_jobconf = super(MRMostUsedWord, self).jobconf()        
        custom_jobconf = {  #key value pairs
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k2,2nr',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        # sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)

    def steps(self):  #pipeline of Map-Reduce jobs
        return [
            MRStep(mapper=self.mapper_get_words,       # STEP 1: word count step
                    combiner=self.combiner_count_words,
                    reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word) # Step 2: most frequent word
        ]

if __name__ == '__main__':
    MRMostUsedWord.run()

Overwriting Mostused.py


Above code has two mapreduce. The first Mapreduce Mapper outputs (word, 1) key value pairs, and then the combiner combines the sum locally. Lastly, Reducer sums them up. The second mapreduce output the word having max number of count.

# Run the code in command line

In [18]:
!python Mostused.py MostUsedWord.txt -r hadoop \
--hadoop-home '/usr/local/Cellar/hadoop/2*/libexec/etc/hadoop/' \
--hadoop-streaming-jar '/usr/local/Cellar/hadoop/2.*/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar' 
--file 

using configs in /Users/leiyang/.mrjob.conf
creating tmp directory /var/folders/tx/5ldq67q511q8wqwqkvptnxd00000gn/T/Mostused.leiyang.20160206.052002.975302
writing wrapper script to /var/folders/tx/5ldq67q511q8wqwqkvptnxd00000gn/T/Mostused.leiyang.20160206.052002.975302/setup-wrapper.sh
Traceback (most recent call last):
  File "Mostused.py", line 53, in <module>
    MRMostUsedWord.run()
  File "/Users/leiyang/anaconda/lib/python2.7/site-packages/mrjob/job.py", line 461, in run
    mr_job.execute()
  File "/Users/leiyang/anaconda/lib/python2.7/site-packages/mrjob/job.py", line 479, in execute
    super(MRJob, self).execute()
  File "/Users/leiyang/anaconda/lib/python2.7/site-packages/mrjob/launch.py", line 153, in execute
    self.run_job()
  File "/Users/leiyang/anaconda/lib/python2.7/site-packages/mrjob/launch.py", line 216, in run_job
    runner.run()
  File "/Users/leiyang/anaconda/lib/python2.7/site-packages/mrjob/runner.py", line 470, in run
    self._run()
  File "/Users/leiyang

# Run the code though python driver

In [13]:
from Mostused import MRMostUsedWord
mr_job = MRMostUsedWord(args=['MostUsedWord.txt'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

(2, 'new')
